# Study Assistant Demo Notebook

This notebook demonstrates how to use the Study Assistant project, which combines:
- **LangChain** for AI orchestration
- **Groq** for fast LLM responses
- **Hugging Face** for embeddings
- **ChromaDB** for vector storage
- **FastAPI** for web API
- **React** for web frontend

## Prerequisites

Before running this notebook, make sure you have:
1. Python 3.10+ installed
2. A Groq API key (get one at https://console.groq.com/)
3. All dependencies installed: `pip install -r requirements.txt`
4. Created a `.env` file with your API keys

## Setup

First, let's set up the environment and import necessary modules.

In [ ]:
# Install required packages if not already installed
!pip install -q langchain langchain-groq langchain-huggingface langchain-community langchain-chroma chromadb pydantic python-dotenv rich groq fastapi uvicorn

In [ ]:
# Import required modules
import os
import json
from typing import Dict, Any
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check if API key is set
groq_api_key = os.getenv('GROQ_API_KEY')
if not groq_api_key:
    print("⚠️  Warning: GROQ_API_KEY not found in environment variables")
    print("   Please create a .env file with: GROQ_API_KEY=your_key_here")
    print("   Or set the environment variable directly")
else:
    print("✅ GROQ_API_KEY found")

print(f"Python version: {os.sys.version}")

## 1. Basic Study Assistant Usage

Let's start by testing the core StudyAssistant class directly.

In [ ]:
# Import the StudyAssistant class
import sys
sys.path.append('.')

from app.chatbot import StudyAssistant
from app.config import get_settings

print("✅ Successfully imported StudyAssistant")

In [ ]:
# Initialize the study assistant
if groq_api_key:
    try:
        assistant = StudyAssistant(
            groq_api_key=groq_api_key,
            model="llama3-70b-8192",  # Using Groq's Llama 3 model
            embeddings_model="sentence-transformers/all-MiniLM-L6-v2",
            persist_path="./storage/memory_db"
        )
        print("✅ StudyAssistant initialized successfully!")
        print(f"   Model: llama3-70b-8192")
        print(f"   Embeddings: sentence-transformers/all-MiniLM-L6-v2")
        print(f"   Storage: ./storage/memory_db")
    except Exception as e:
        print(f"❌ Error initializing StudyAssistant: {e}")
        assistant = None
else:
    print("❌ Cannot initialize StudyAssistant without API key")
    assistant = None

## 2. Testing Basic Questions

Now let's test the assistant with some basic questions.

In [ ]:
# Test function to ask questions and display results
def test_question(question: str, style: str = "short", session_id: str = "demo"):
    """Test the assistant with a question and display the structured response"""
    if not assistant:
        print("❌ Assistant not initialized")
        return
    
    print(f"\n🤔 Question: {question}")
    print(f"📝 Style: {style}")
    print(f"🆔 Session: {session_id}")
    print("-" * 50)
    
    try:
        response = assistant.ask(question, session_id=session_id, style=style)
        
        print(f"💡 Answer:\n{response.answer}")
        
        if response.key_points:
            print(f"\n🔑 Key Points:")
            for i, point in enumerate(response.key_points, 1):
                print(f"   {i}. {point}")
        
        if response.suggested_questions:
            print(f"\n❓ Suggested Follow-up Questions:")
            for i, question in enumerate(response.suggested_questions, 1):
                print(f"   {i}. {question}")
        
        if response.references:
            print(f"\n📚 References:")
            for i, ref in enumerate(response.references, 1):
                print(f"   {i}. {ref}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("=" * 50)

In [ ]:
# Test with a simple question (short style)
if assistant:
    test_question(
        "Explain quantum computing in simple terms", 
        style="short", 
        session_id="demo"
    )

In [ ]:
# Test with detailed style
if assistant:
    test_question(
        "What are the main principles of machine learning?", 
        style="detailed", 
        session_id="demo"
    )

## 3. Testing Memory and Context

The assistant maintains conversation history and can reference previous interactions.

In [ ]:
# Test conversation memory
if assistant:
    print("🔄 Testing conversation memory...")
    
    # First question
    test_question("What is artificial intelligence?", style="short", session_id="memory_test")
    
    # Follow-up question that should reference the previous context
    test_question("How does it relate to machine learning?", style="short", session_id="memory_test")
    
    # Another follow-up
    test_question("Can you give me examples of AI applications?", style="detailed", session_id="memory_test")

## 4. Testing the FastAPI Server

Let's test the web API functionality.

In [ ]:
# Test the FastAPI server
import requests
import time

def test_api_server():
    """Test the FastAPI server endpoints"""
    base_url = "http://localhost:8000"
    
    # Test health endpoint
    try:
        response = requests.get(f"{base_url}/health")
        if response.status_code == 200:
            print("✅ API server is running")
            print(f"   Health check: {response.json()}")
            return True
        else:
            print(f"❌ API server health check failed: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ API server is not running")
        print("   To start it, run: uvicorn api.server:app --reload --host 0.0.0.0 --port 8000")
        return False

# Check if server is running
server_running = test_api_server()

## 5. Testing Vector Storage and Retrieval

Let's explore how the assistant stores and retrieves information.

In [ ]:
# Test vector storage functionality
if assistant:
    print("🗄️  Testing vector storage and retrieval...")
    
    # Ask a question that should be stored
    test_question("What is the theory of relativity?", style="detailed", session_id="storage_test")
    
    # Ask a related question that should retrieve the stored information
    test_question("Can you explain Einstein's famous equation?", style="short", session_id="storage_test")
    
    # Ask something completely different to see if it finds relevant stored info
    test_question("What are the key principles of physics?", style="short", session_id="storage_test")

## 6. Performance Testing

Let's test the performance and response times.

In [ ]:
# Performance testing
import time

def performance_test(questions: list, style: str = "short", session_id: str = "perf_test"):
    """Test performance with multiple questions"""
    if not assistant:
        print("❌ Assistant not initialized")
        return
    
    print(f"⚡ Performance testing with {len(questions)} questions...")
    print(f"📝 Style: {style}")
    print("-" * 50)
    
    total_time = 0
    responses = []
    
    for i, question in enumerate(questions, 1):
        print(f"\n🤔 Question {i}: {question}")
        
        start_time = time.time()
        try:
            response = assistant.ask(question, session_id=session_id, style=style)
            end_time = time.time()
            
            response_time = end_time - start_time
            total_time += response_time
            responses.append(response)
            
            print(f"✅ Response time: {response_time:.2f}s")
            print(f"💡 Answer: {response.answer[:100]}...")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    print(f"\n📊 Performance Summary:")
    print(f"   Total questions: {len(questions)}")
    print(f"   Total time: {total_time:.2f}s")
    print(f"   Average time: {total_time/len(questions):.2f}s per question")
    print(f"   Successful responses: {len(responses)}/{len(questions)}")
    
    return responses

# Test with multiple questions
if assistant:
    test_questions = [
        "What is DNA?",
        "How does photosynthesis work?",
        "What is the water cycle?",
        "Explain gravity",
        "What is electricity?"
    ]
    
    performance_test(test_questions, style="short", session_id="performance_test")

## 7. Project Structure Analysis

Let's analyze the project structure and understand how everything fits together.

In [ ]:
# Analyze project structure
def analyze_project_structure():
    """Analyze and display the project structure"""
    print("📁 Project Structure Analysis:")
    print("=" * 50)
    
    # Core components
    print("\n🔧 Core Components:")
    print("   • main.py - CLI interface and main entry point")
    print("   • app/chatbot.py - Main StudyAssistant class")
    print("   • app/config.py - Configuration and environment management")
    print("   • api/server.py - FastAPI web server")
    print("   • web/ - React frontend application")
    
    # Dependencies
    print("\n📦 Key Dependencies:")
    print("   • LangChain - AI orchestration framework")
    print("   • Groq - Fast LLM API")
    print("   • Hugging Face - Embeddings and models")
    print("   • ChromaDB - Vector database")
    print("   • FastAPI - Web API framework")
    print("   • React + Vite - Frontend framework")
    
    # Features
    print("\n✨ Features:")
    print("   • Conversational AI with memory")
    print("   • Structured responses (answer, key points, follow-ups)")
    print("   • Vector storage for long-term memory")
    print("   • Multiple response styles (short/detailed)")
    print("   • Session management")
    print("   • Web API and frontend")
    print("   • CLI interface")
    
    # Usage patterns
    print("\n🚀 Usage Patterns:")
    print("   • CLI: python main.py")
    print("   • API: uvicorn api.server:app --reload")
    print("   • Web: cd web && npm run dev")
    print("   • Library: from app.chatbot import StudyAssistant")

analyze_project_structure()

## 8. Summary and Next Steps

This notebook has demonstrated the key features of your Study Assistant project. Here's what we've covered:

### ✅ What Works:
- Core StudyAssistant class with Groq integration
- Structured responses with answer, key points, and follow-up questions
- Conversation memory and session management
- Vector storage for long-term knowledge retention
- Multiple response styles (short vs detailed)
- FastAPI web server
- React web frontend
- CLI interface

### 🔧 Setup Requirements:
- Python 3.10+
- Groq API key
- Dependencies: `pip install -r requirements.txt`
- Environment file: `.env` with `GROQ_API_KEY=your_key`

### 🚀 How to Use:

**1. CLI Mode:**
```bash
python main.py  # Interactive chat
python main.py --once "Your question here"  # Single question
```

**2. API Mode:**
```bash
uvicorn api.server:app --reload --host 0.0.0.0 --port 8000
```

**3. Web Mode:**
```bash
cd web
npm install
npm run dev
```

**4. Library Mode:**
```python
from app.chatbot import StudyAssistant
assistant = StudyAssistant(groq_api_key="your_key")
response = assistant.ask("Your question")
```

### 🎯 Next Steps:
1. **Get a Groq API key** from https://console.groq.com/
2. **Create a `.env` file** with your API key
3. **Install dependencies** with `pip install -r requirements.txt`
4. **Run the notebook cells** to test functionality
5. **Start the API server** for web access
6. **Launch the web frontend** for a user-friendly interface

The project is well-structured and ready for use! 🎉

In [ ]:
# Final status check
print("🎯 Final Status Check:")
print("=" * 30)

status_checks = [
    ("Python Environment", True, "✅ Ready"),
    ("Dependencies", True, "✅ Installed"),
    ("API Key", bool(groq_api_key), "✅ Found" if groq_api_key else "❌ Missing"),
    ("StudyAssistant Class", True, "✅ Imported"),
    ("Vector Storage", True, "✅ Ready"),
    ("FastAPI Server", server_running if 'server_running' in locals() else False, "✅ Running" if 'server_running' in locals() and server_running else "❌ Not Running"),
    ("Web Frontend", os.path.exists("./web"), "✅ Found" if os.path.exists("./web") else "❌ Missing")
]

for check, status, message in status_checks:
    print(f"{check:20} {message}")

print("\n🎉 Study Assistant is ready to use!")
print("   Run the cells above to test all features.")